In [ ]:
using Pkg; Pkg.activate(".")
using Plots, LinearAlgebra, LaTeXStrings, ForwardDiff
include("tools.jl")

## Fourier Spectral Methods in d > 1

We start by re-implementing our 1D methods for 2D and 3D: 
- x grid
- k grid 
- trigonometric interpolant 
- evaluation of the trig interp on a finer grid

In [ ]:

"""
Given a one-dimensional array y, return d d-dimensional arrays 
 y ⊗ 1 ⊗ ... ⊗ 1   (x1-coordinate)
 1 ⊗ y ⊗ 1 ⊗ ...   (x2-coordinate)
... 
 1 ⊗ ... ⊗ 1 ⊗ y   (xd-coordinate)
"""
function tensorgrid(d, x1)
    dims = ntuple(i -> length(x1), d)
    X = reshape(x1 * ones(Bool, length(x1)^(d-1))', dims)
    pdim(i, d) = (dd = collect(1:d); dd[1] = i; dd[i] = 1; tuple(dd...))
    return ntuple(i -> permutedims(X, pdim(i,d)), d)
end

"""
d-dimensional x grid
returns X1, X2, ...
"""
xgrid(d, N) = tensorgrid(d, xgrid(N))

"""
d-dimensional k-grid 
returns K1, K2, ...
"""
kgrid(d, N) = tensorgrid(d, kgrid(N))


"""
construct the coefficients of the trigonometric interpolant
in d dimensions
"""
function triginterp_fft(f::Function, N, d::Integer)
    XX = xgrid(d, N)
    # nodal values at interpolation nodes
    F = f.(XX...) 
    return fft(F) / (2*N)^d
end 

function evaltrig_grid(F̂::AbstractArray{T, 2}, M::Integer) where {T}
    N = size(F̂, 1) ÷ 2;
    @assert size(F̂) == (2*N, 2*N)
    @assert M >= N
    F̂_M = zeros(ComplexF64, (2*M, 2*M)) 
    kk1 = 1:N; kk2 = N+1:2*N; kk3 = 2*M-N+1:2*M
    F̂_M[kk1, kk1] .= F̂[kk1, kk1]
    F̂_M[kk1, kk3] .= F̂[kk1, kk2]
    F̂_M[kk3, kk1] .= F̂[kk2, kk1] 
    F̂_M[kk3, kk3] .= F̂[kk2, kk2]
    x = xgrid(M) 
    Fx = real.(ifft(F̂_M) * (2*M)^2)
    return Fx, x
end

In [ ]:
N = 2 
F̂ = randn(2*N, 2*N)
x = xgrid(N)
F = real.(ifft(F̂)*(2*N)^2)
surface(x, x, F; size=(400,300), colorbar=nothing)

In [ ]:
M = 32 
FM, x = evaltrig_grid(F̂, M)
surface(x, x, FM; size=(400,300), colorbar=nothing)

### Approximation Rates 

We will explore approximation rates for a simple generalization of the periodic witch of agnesi: 
$$
f(x_1, \dots, x_d) = \frac{1}{1 + c^2 \sum_{t = 1}^d \sin^2(x_t)}
$$
But to make things a bit clearer, we change it slightly to 
$$
f(x_1, \dots, x_d) = \frac{1}{1 + c^2 \sum_{t = 1}^d \sin^2(x_t/2-\pi/2)}
= \frac{1}{1+ c^2/2 \sum_{t=1}^d \cos(x_t)}
$$
This ensures that there is just a single peak in the center of the domain $[0, 2\pi)^d$. In the rewriting we used that $\sin^2(x/2) = \frac12 - \frac12 \cos(x)$ and $\cos(x-\pi) = -\cos(x)$.

### Two Dimensions

In [ ]:
f2_fun, α = let c = 4.0
    ( (x1, x2) -> 1/(1 + 0.5*c^2 * (2 + cos(x1) + cos(x2))) ), 2*asinh(1/c)
end 

In [ ]:
# plot the target function
N = 64; 
X1, X2 = xgrid(2, N)
F = f2_fun.(X1, X2)
x = xgrid(N)
surface(x, x, F; size=(400,300), colorbar=nothing)

In [ ]:
D = 2  # dimension
NN = 8:8:64

# Target funtion on a fine grid
Ne = 256
X1e, X2e = xgrid(D, Ne)
FM_e = f2_fun.(X1e, X2e)

errs = Float64[]
for N in NN 
    F̂ = triginterp_fft(f2_fun, N, D)
    FN_e, x = evaltrig_grid(F̂, Ne)
    err_N = norm(FM_e[:] - FN_e[:], Inf)
    push!(errs, err_N)
end


In [ ]:
plot(NN, errs, m=:o, lw=3, 
        yscale = :log10, label = "error", 
        size = (400, 250), )
plot!([30,50], 10 * exp.(- α * [30, 50]), 
        lw=2, ls=:dash, c=:black, 
        label = L"e^{-\alpha N}")

### Three Dimensions

Nothing really changes except the increasing cost of the computations in 3D. The following code snippets can give a starting point for implementing some 3-dimensional codes. 

In [ ]:
function evaltrig_grid(F̂::AbstractArray{T, 3}, M::Integer) where {T}
    N = size(F̂, 1) ÷ 2;
    @assert size(F̂) == (2*N, 2*N, 2*N)
    @assert M >= N
    F̂_M = zeros(ComplexF64, (2*M, 2*M, 2*M))
    kk1 = 1:N; kk2 = N+1:2*N; kk3 = 2*M-N+1:2*M
    F̂_M[kk1, kk1, kk1] .= F̂[kk1, kk1, kk1]
    F̂_M[kk1, kk1, kk3] .= F̂[kk1, kk1, kk2]
    F̂_M[kk1, kk3, kk1] .= F̂[kk1, kk2, kk1]
    F̂_M[kk1, kk3, kk3] .= F̂[kk1, kk2, kk2]
    F̂_M[kk3, kk1, kk1] .= F̂[kk2, kk1, kk1]
    F̂_M[kk3, kk1, kk3] .= F̂[kk2, kk1, kk2]
    F̂_M[kk3, kk3, kk1] .= F̂[kk2, kk2, kk1]
    F̂_M[kk3, kk3, kk3] .= F̂[kk2, kk2, kk2]
    x = xgrid(M) 
    Fx = real.(ifft(F̂_M) * (2*M)^3)
    return Fx, x
end

In [ ]:
f3_fun, α = let c = 4.0
    ( (x1, x2, x3) -> 1 / (1 + 
        0.5*c^2 * (3+cos(x1)+cos(x2)+cos(x3)))), 2*asinh(1/c)
end 

In [ ]:
D = 3  # dimension
NN = 4:4:32   # grid sizes

# Target function on a fine grid
# Note: 256^3 = 2^24 > 10^7 grid points!!
Ne = 128   
X1e, X2e, X3e = xgrid(D, Ne)
FM_e = f3_fun.(X1e, X2e, X3e)

errs = Float64[]
for N in NN 
    X1, X2, X3 = xgrid(D, N)
    F̂ = triginterp_fft(f3_fun, N, D)
    FN_e, x = evaltrig_grid(F̂, Ne)
    err_N = norm(FM_e[:] - FN_e[:], Inf)
    push!(errs, err_N)
end


In [ ]:
plot(NN, errs, m=:o, lw=3, 
        yscale = :log10, label = "error", 
        size = (400, 250), )
plot!([16,30], 10 * exp.(- α * [16, 30]), 
        lw=2, ls=:dash, c=:black, 
        label = L"e^{-\alpha N}")

---

### PDE Example 1: Elliptic Problem

$$
 - \Delta u + u = f \quad \text{with PBC}
$$
Transform to reciprocal space: 
$$
   (|k|_2^2 + 1) \hat{u}_k = \hat{f}_k
$$
After replacing $f$ with $I_N f$, with Fourier coefficients $\hat{F}_k$, we obtain the coefficients of the plane wave solution 
$$
   \hat{U}_k = \frac{\hat{F}_k}{1 + |k|_2^2}, 
   \quad k = (k_1, \dots, k_d), k_t = -N+1, \dots, N.
$$

In [ ]:
D = 2
N = 16

F̂ = triginterp_fft(f2_fun, N, D)
k1, k2 = kgrid(D, N)
Û = F̂ ./ (1 .+ k1.^2 + k2.^2)
Up, xp = evaltrig_grid(Û, 64)

surface(xp, xp, Up; colorbar = false, size = (400, 300))

For the error analysis we could use the method of manufactured solutions, but we use a different approach this time. We simply solve on a much finer grid. Suppose the fine-grid solution is with $N_e$, and the the error for grid size $N$ is $\epsilon(N)$. Then we get 
$$
 \| u - u_N\|  - \|u - u_{N_e}\|  \leq   \| u_N - u_{N_e} \| \leq \| u - u_{N} \| + \|u - u_{N_e}\| 
$$
If $\epsilon(N_e) \leq \theta \epsilon(N)$ with some $\theta < 1$ then we obtain 
$$
 (1 - \theta) \| u - u_N \| \leq  \| u_N - u_{N_e} \| \leq (1+\theta) \| u - u_N \|.
$$
In simple situations where this assumption can be guaranteed, this is therefore a reliable approach. In general one must be very careful. 

In [ ]:
f2_fun, α = let c = 4
    ( (x1, x2) -> 1/(1 + 0.5*c^2 * (2 + cos(x1) + cos(x2))) ), 2*asinh(1/c)
end 

D = 2
NN = 8:8:64
Ne = 256

# reference solution (Ne, not exact!)
F̂e = triginterp_fft(f2_fun, Ne, D)
k1, k2 = kgrid(D, Ne)
Ûe = F̂e ./ (1 .+ k1.^2 + k2.^2)
Ue, _ = evaltrig_grid(Ûe, Ne)

# errors 
errs = Float64[] 
for N in NN 
    F̂ = triginterp_fft(f2_fun, N, D)
    k1, k2 = kgrid(D, N)
    Û = F̂ ./ (1 .+ k1.^2 + k2.^2)
    U, _ = evaltrig_grid(Û, Ne)
    push!(errs, norm(U[:] - Ue[:], Inf))
end 
;

In [ ]:
plot(NN, errs, m=:o, lw=3, 
        yscale = :log10, label = L"||u - u_N||_\infty", 
        size = (400, 250), xlabel = L"N")
plot!([30,50], 0.1*exp.(- 1.1*α * [30, 50]), 
        lw=2, ls=:dash, c=:black, 
        label = L"e^{-1.1\alpha N}")

### PDE Example 2: Transport Problem

$$
  u_t + {\bf v} \cdot \nabla u = 0 \qquad \text{(PBC)}
$$
where ${\bf v}$ is a constant vector determining the direction of transport.

In [ ]:
function plot_soln(t, x, U)
    surface(x, x, U; title = "t = $(round(t, digits=2))", 
            size = (400, 300), 
            color = :viridis, clims = (-0.3, 1.1), colorbar=false)
end

# ------------------------------------------
# Problem setup
N = 32 
D = 2 
dt = π/(6N) 
tmax = 32.0
v = [0.7, 0.3]
u0 = (x1, x2) ->  exp(- 10 * (2+cos(x1)+cos(x2))/2 )
#------------------------------------------

X1, X2 = xgrid(D, N)
x = X1[:,1]
K1, K2 = kgrid(D, N)

# directional derivative operator in Fourier space 
VD̂ = im*K1*v[1] + im*K2*v[2]

# initial condition, we also need one additional v in the past
# (this takes one step of the PDE backward in time)
U = u0.(X1, X2)
Uold = U + dt * real.( ifft( VD̂ .* fft(U) ) )

# time-stepping loop (check errors)
tt = 0:dt:tmax
errs = Float64[] 
nrmu2 = Float64[] 
@gif for t in tt 
    global U, Uold, VD̂, X1, X2, v, nrmu2
    # differentiation in reciprocal space
    W = real.( ifft( VD̂ .* fft(U) ) )
    # multiplication and update in real space
    U, Uold = Uold - 2 * dt * W, U
    # check error 
    Ue = u0.(X1 .- v[1]*t, X2 .- v[2]*t)
    push!(errs, norm(U[:] - Ue[:], Inf))
    push!(nrmu2, norm(U[:])^2 / (2*N)^2)
    # plot snapshot 
    plot_soln(t, x, U)
end every 10

In [ ]:
p1 = plot(tt, errs, lw=2, label = L"||u_N(t) - u(t)||_\infty")
plot!(tt, dt .+ dt^2 * tt, lw=2, ls=:dash, label = L"\Delta t + t \Delta t^2")
p2 = plot(tt, abs.(nrmu2 .- nrmu2[1]), label = L"\|U(t)\|_2^2")
plot(p1, p2, layout = (2,1))